<a id='1.3'></a>
## Import Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor
from sklearn.preprocessing import StandardScaler #平均值是0, 標準差是1
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint
import sys
sys.path.append("../FinRL")
import itertools

c:\Users\shig3\miniconda3\envs\fintech\lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


<a id='1.4'></a>
## Create Folders

In [3]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
    CHIPS,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

In [4]:
TRAIN_START_DATE = '2015-01-01'
TRAIN_END_DATE = '2022-11-01'
TRADE_START_DATE = '2022-11-01'
TRADE_END_DATE = '2024-05-20'
PREDICT_START_DATE = '2025-05-01'
PREDICT_END_DATE = '2025-05-25'

In [5]:
df = pd.read_csv('Input for Stock-Picked Agent train.csv')
df = df.fillna(0)

In [6]:
features = ['CostOfGoodsSold',
            'EPS',
            'EquityAttributableToOwnersOfParent_x',
            'IncomeAfterTaxes',
            'IncomeFromContinuingOperations',
            'OtherComprehensiveIncome',
            'Revenue',
            'TAX',
            'TotalConsolidatedProfitForThePeriod',	
            'CapitalStock',
            'CapitalStock_per',	
            'CapitalSurplus',
            'CapitalSurplus_per',	
            'CashAndCashEquivalents',	
            'CashAndCashEquivalents_per',	
            'CurrentAssets',
            'CurrentAssets_per',	
            'Equity',
            'EquityAttributableToOwnersOfParent_y',
            'EquityAttributableToOwnersOfParent_per',	
            'Equity_per',
            'NoncurrentAssets',
            'NoncurrentAssets_per',	
            'NoncurrentLiabilities',
            'NoncurrentLiabilities_per',	
            'OrdinaryShare',
            'OrdinaryShare_per',	
            'OtherCurrentLiabilities',	
            'OtherCurrentLiabilities_per',	
            'OtherEquityInterest',
            'OtherEquityInterest_per',	
            'RetainedEarnings',
            'RetainedEarnings_per',	
            'TotalAssets',
            'TotalAssets_per',	
            'CashBalancesBeginningOfPeriod',
            'CashBalancesEndOfPeriod',
            'Depreciation',
            'PayTheInterest',	
            'PropertyAndPlantAndEquipment' ]

# 建立 StandardScaler 物件
ss = StandardScaler()

df_features_scaled = pd.DataFrame(ss.fit_transform(df[features]), columns=features, index=df.index)


df.update(df_features_scaled)

In [7]:
fe = FeatureEngineer(
                    use_technical_indicator=False,
                    tech_indicator_list = INDICATORS,
                    use_vix=False,
                    use_turbulence=False, 
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

In [8]:
processed

,date,tic,CostOfGoodsSold,EPS,EquityAttributableToOwnersOfParent_x,IncomeAfterTaxes,IncomeFromContinuingOperations,OtherComprehensiveIncome,Revenue,TAX,...,4938,4958,5871,6505,8046,9904,9910,9921,9941,9945
0,2015-05-16,1101.TW,-0.277418,-0.419264,-0.229072,-0.235074,-0.235231,-0.129758,-0.306118,-0.210838,...,0.033775,0.000000,0.000000,0.000000,0.000000,0.033798,0.000000,0.000000,0.000000,0.000000
0,2015-05-16,1102.TW,-0.307297,-0.377587,-0.198536,-0.208264,-0.208420,0.676541,-0.340500,-0.263045,...,0.000000,0.000000,0.033780,0.033780,0.033780,0.000000,0.000000,0.000000,0.000000,0.000000
0,2015-05-16,1216.TW,0.041902,-0.324240,-0.067496,0.031326,0.031177,-0.136264,0.166243,0.129428,...,0.000000,0.000000,0.033782,0.000000,0.033782,0.000000,0.000000,0.000000,0.000000,0.000000
0,2015-05-16,1229.TW,-0.385132,-0.400926,-0.253478,-0.267751,-0.267909,-0.076218,-0.420313,-0.343156,...,0.033775,0.033775,0.000000,0.000000,0.000000,0.000000,0.000000,0.033775,0.033775,0.000000
0,2015-05-16,1301.TW,-0.132335,-0.360916,-0.097850,-0.109037,-0.109191,0.495958,-0.157582,-0.126158,...,0.033782,0.000000,0.000000,0.000000,0.033782,0.033787,0.033782,0.033782,0.033782,0.033782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,2025-05-16,9904.TW,-0.075076,-0.252555,-0.227558,-0.043277,-0.043428,-0.246265,-0.048214,-0.015407,...,0.000000,0.000000,0.000000,0.033815,0.000000,0.000000,0.000000,0.000000,0.000000,0.033785
40,2025-05-16,9910.TW,-0.291169,-0.262558,-0.198043,-0.224602,-0.224759,0.063816,-0.310176,-0.221112,...,0.000000,0.033798,0.000000,0.000000,0.000000,0.033798,0.033725,0.000000,0.000000,0.000000
40,2025-05-16,9921.TW,-0.306532,-0.299234,-0.249494,-0.264297,-0.264455,0.116730,-0.332080,-0.308573,...,0.000000,0.033786,0.000000,0.000000,0.000000,0.033786,0.033765,0.033795,0.000000,0.033786
40,2025-05-16,9941.TW,-0.359511,-0.124189,-0.207882,-0.227788,-0.227945,0.023320,-0.372541,-0.249155,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033807,0.000000,0.000000


## Data Split

In [9]:
train = data_split(processed, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed, TRADE_START_DATE,TRADE_END_DATE)
PREDICT = data_split(processed, PREDICT_START_DATE,PREDICT_END_DATE)

print(len(train))
print(len(trade))
print(len(PREDICT))

2220
518
74


In [14]:
stock_dimension = len(train.tic.unique()) 
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension 
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 74, State Space: 8585


In [16]:
buy_cost_list =  [0] * stock_dimension
sell_cost_list = [0] * stock_dimension
buy_cost_list =  [0] * stock_dimension
sell_cost_list = [0] * stock_dimension
num_stock_shares = [0] * stock_dimension


env_kwargs = {
    "hmax": 1,
    "initial_amount": 0,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 0
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)


In [17]:
env_train, _ = e_train_gym.get_sb_env()

c:\Users\shig3\miniconda3\envs\fintech\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [18]:
agent = DRLAgent(env = env_train)

### Agent: PPO

In [19]:
df_account_value_ppo = pd.DataFrame()

agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 2048,
}
#model_ppo = agent.get_model("ppo")
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)
# set up logger
tmp_path = RESULTS_DIR + '/ppo'
new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
# Set new logger
model_ppo.set_logger(new_logger_ppo)

trained_ppo = agent.train_model(model=model_ppo, 
                         tb_log_name='ppo',
                         total_timesteps=3440)  

Get Model kwargs:  {'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 2048}
Using cpu device
Logging to results/ppo
-----------------------------------
| time/              |            |
|    fps             | 22         |
|    iterations      | 1          |
|    time_elapsed    | 92         |
|    total_timesteps | 2048       |
| train/             |            |
|    reward          | 0.04781683 |
-----------------------------------
day: 29, episode: 100
begin_total_asset: 0.00
end_total_asset: 0.71
total_reward: 0.71
total_cost: 0.00
total_trades: 1091
Sharpe: nan
-----------------------------------------
| time/                   |             |
|    fps                  | 21          |
|    iterations           | 2           |
|    time_elapsed         | 189         |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.00911911  |
|    clip_fraction        | 0.103       |
|    clip_range       

In [20]:
trained_ppo.save("PPO_Stock_Picked_Agent_Sharpe_Ratio.zip")

In [21]:
trade

,date,tic,CostOfGoodsSold,EPS,EquityAttributableToOwnersOfParent_x,IncomeAfterTaxes,IncomeFromContinuingOperations,OtherComprehensiveIncome,Revenue,TAX,...,4938,4958,5871,6505,8046,9904,9910,9921,9941,9945
0,2022-11-15,1101.TW,-0.214266,-0.419264,-0.222312,-0.203716,-0.203873,-0.069337,-0.251342,-0.219123,...,0.033784,0.000000,0.000000,0.000000,0.000000,0.033784,0.033784,0.000000,0.033784,0.033784
0,2022-11-15,1102.TW,-0.273991,-0.234217,-0.062123,-0.067919,-0.068071,0.453681,-0.303375,-0.184857,...,0.000000,0.000000,0.000000,0.033784,0.000000,0.033784,0.033784,0.033784,0.033784,0.033784
0,2022-11-15,1216.TW,0.202332,-0.294233,0.153974,0.118605,0.118458,0.560493,0.374544,0.440899,...,0.000000,0.033784,0.033784,0.033784,0.000000,0.033784,0.000000,0.000000,0.000000,0.033784
0,2022-11-15,1229.TW,-0.377126,-0.397592,-0.236381,-0.250542,-0.250700,-0.118913,-0.412436,-0.343897,...,0.000000,0.000000,0.033784,0.000000,0.033784,0.000000,0.000000,0.033784,0.000000,0.000000
0,2022-11-15,1301.TW,-0.075461,-0.304235,-0.594047,-0.009090,-0.009240,-2.199631,-0.112575,-0.251433,...,0.033784,0.000000,0.033784,0.000000,0.000000,0.000000,0.033784,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,2024-05-16,9904.TW,-0.101333,-0.192540,0.215392,0.026159,0.026009,1.445059,-0.069379,0.031827,...,0.033745,0.000000,0.000000,0.000000,0.000000,0.000000,0.033743,0.033744,0.000000,0.000000
6,2024-05-16,9910.TW,-0.291490,-0.240886,-0.207693,-0.217834,-0.217990,0.182720,-0.310778,-0.213447,...,0.033745,0.033745,0.000000,0.000000,0.033752,0.033745,0.000000,0.000000,0.033766,0.033967
6,2024-05-16,9921.TW,-0.312864,-0.234217,-0.242452,-0.253656,-0.253814,0.098054,-0.336536,-0.279543,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033766,0.033968
6,2024-05-16,9941.TW,-0.361208,-0.077511,-0.188628,-0.218338,-0.218495,0.084391,-0.368591,-0.231357,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.033745,0.000000,0.033745,0.033767,0.000000


In [22]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = None,risk_indicator_col=None, **env_kwargs)

trained_moedl =trained_ppo
df_account_value, df_actions ,df_balance= DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)

print("==============Get Backtest Results===========")

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all.csv")


c:\Users\shig3\miniconda3\envs\fintech\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


hit end!
==============Get Backtest Results===========
Annual return          inf
Cumulative returns     inf
Annual volatility      NaN
Sharpe ratio           NaN
Calmar ratio           NaN
Stability              NaN
Max drawdown           0.0
Omega ratio            inf
Sortino ratio          inf
Skew                   NaN
Kurtosis               NaN
Tail ratio             NaN
Daily value at risk    NaN
dtype: float64


c:\Users\shig3\miniconda3\envs\fintech\lib\site-packages\numpy\lib\nanfunctions.py:1734: RuntimeWarning: invalid value encountered in subtract
  np.subtract(arr, avg, out=arr, casting='unsafe', where=where)
c:\Users\shig3\miniconda3\envs\fintech\lib\site-packages\numpy\lib\nanfunctions.py:1734: RuntimeWarning: invalid value encountered in subtract
  np.subtract(arr, avg, out=arr, casting='unsafe', where=where)
c:\Users\shig3\miniconda3\envs\fintech\lib\site-packages\empyrical\stats.py:396: RuntimeWarning: invalid value encountered in subtract
  nanmin((cumulative - max_return) / max_return, axis=0, out=out)
c:\Users\shig3\miniconda3\envs\fintech\lib\site-packages\numpy\lib\function_base.py:2699: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]
c:\Users\shig3\miniconda3\envs\fintech\lib\site-packages\empyrical\stats.py:396: RuntimeWarning: invalid value encountered in subtract
  nanmin((cumulative - max_return) / max_return, axis=0, out=out)
c:\Users\shig3\minic

In [23]:
PREDICT

,date,tic,CostOfGoodsSold,EPS,EquityAttributableToOwnersOfParent_x,IncomeAfterTaxes,IncomeFromContinuingOperations,OtherComprehensiveIncome,Revenue,TAX,...,4938,4958,5871,6505,8046,9904,9910,9921,9941,9945
0,2025-05-16,1101.TW,-0.212823,-0.444271,0.064601,-0.244807,-0.244964,1.566491,-0.229658,-0.040789,...,0.033785,0.000000,0.000000,0.033815,0.033766,0.000000,0.033764,0.000000,0.033779,0.033785
0,2025-05-16,1102.TW,-0.304263,-0.387590,-0.203358,-0.215618,-0.215775,0.504563,-0.333529,-0.243798,...,0.000000,0.033800,0.000000,0.000000,0.033718,0.000000,0.033716,0.033818,0.033768,0.000000
0,2025-05-16,1216.TW,0.305165,-0.304235,-0.027806,0.108110,0.107963,0.443138,0.530130,0.488050,...,0.000000,0.033797,0.033734,0.033827,0.000000,0.033797,0.033729,0.000000,0.000000,0.033797
0,2025-05-16,1229.TW,-0.379259,-0.364250,-0.330251,-0.237262,-0.237419,-0.541885,-0.413681,-0.336873,...,0.000000,0.033785,0.033769,0.033815,0.000000,0.000000,0.033764,0.000000,0.000000,0.033785
0,2025-05-16,1301.TW,-0.116212,-0.454273,-0.060002,-0.278708,-0.278866,0.759515,-0.160382,-0.328481,...,0.000000,0.033785,0.000000,0.000000,0.000000,0.000000,0.033764,0.033803,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2025-05-16,9904.TW,-0.075076,-0.252555,-0.227558,-0.043277,-0.043428,-0.246265,-0.048214,-0.015407,...,0.000000,0.000000,0.000000,0.033815,0.000000,0.000000,0.000000,0.000000,0.000000,0.033785
0,2025-05-16,9910.TW,-0.291169,-0.262558,-0.198043,-0.224602,-0.224759,0.063816,-0.310176,-0.221112,...,0.000000,0.033798,0.000000,0.000000,0.000000,0.033798,0.033725,0.000000,0.000000,0.000000
0,2025-05-16,9921.TW,-0.306532,-0.299234,-0.249494,-0.264297,-0.264455,0.116730,-0.332080,-0.308573,...,0.000000,0.033786,0.000000,0.000000,0.000000,0.033786,0.033765,0.033795,0.000000,0.033786
0,2025-05-16,9941.TW,-0.359511,-0.124189,-0.207882,-0.227788,-0.227945,0.023320,-0.372541,-0.249155,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033807,0.000000,0.000000


In [24]:
e_predict_gym = StockTradingEnv(df = PREDICT, turbulence_threshold = None,risk_indicator_col=None, **env_kwargs)

predict_env, predict_obs = e_predict_gym.get_sb_env()
predict_env.reset()
action, _states = trained_ppo.predict(predict_obs, deterministic=True)

print(action)

[[1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 1 1 1 0 0 0 0 0 1 0 0 1 0 0 1 1 0 0
  1 0 1 1 1 0 1 1 0 1 1 0 0 1 0 0 0 0 0 1 1 1 0 1 0 1 0 0 0 1 0 1 1 1 0 0
  1 1]]


c:\Users\shig3\miniconda3\envs\fintech\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
